<a href="https://colab.research.google.com/github/martin-fabbri/recsys-pipeline/blob/main/tiktok-pipeline/1_feature_backfill.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## <span style='color:#ff5f27'> 📝 Imports

In [1]:
!rm *.py
!wget https://raw.githubusercontent.com/martin-fabbri/recsys-pipeline/refs/heads/main/tiktok-pipeline/features.py

!pip install -U hopsworks[python] --quiet
!pip install -U mimesis==15.1.0 --quiet
!pip install great-expectations==0.18.21 --quiet

rm: cannot remove '*.py': No such file or directory
--2024-12-09 02:52:17--  https://raw.githubusercontent.com/martin-fabbri/recsys-pipeline/refs/heads/main/tiktok-pipeline/features.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5649 (5.5K) [text/plain]
Saving to: ‘features.py’

features.py         100%[===================>]   5.52K  --.-KB/s    in 0s      

2024-12-09 02:52:17 (56.7 MB/s) - ‘features.py’ saved [5649/5649]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 5.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 3.3 MB/s eta 0:00:00
  

# 🚨🚨🚨 ⚠️ Restart the Colab Session before continuing ⚠️ 🚨🚨🚨

In [2]:
!pip install -U confluent-kafka==2.3.0 --quiet

In [3]:
import pandas as pd
import numpy as np
import great_expectations as ge
from great_expectations.core import ExpectationSuite, ExpectationConfiguration

from features import generate_users
from features import generate_video_content
from features import generate_interactions

## <span style='color:#ff5f27'> ⚙️ Data Generation</span>


### <span style='color:#ff5f27'> 👥 Users Data Generation</span>

In [4]:
user_data = generate_users(25_000)

data_users_df = pd.DataFrame(user_data)
data_users_df.head()

,user_id,gender,age,country
0,FI482L,Female,20,Albania
1,EJ959G,Other,42,Afghanistan
2,JD331A,Other,73,Austria
3,VD034W,Male,60,Seychelles
4,PV828R,Female,72,Malta


### <span style='color:#ff5f27'> 🎥 Content Data Generation</span>


In [5]:
# Generate data for 25_000 videos
video_data = generate_video_content(25_000, historical=True)

data_video_df = pd.DataFrame(video_data)
data_video_df.head()

,video_id,category,views,likes,video_length,upload_date
0,3PM53D,Entertainment,189517,7908,94,2023-12-26
1,8XN69L,Comedy,361021,188591,73,2024-09-03
2,4UA61G,Music,143705,132667,178,2024-06-06
3,0JJ72E,Education,254033,165767,249,2024-02-22
4,3WG38W,Music,6844,2229,205,2023-04-11


### <span style='color:#ff5f27'> 🔗 Interactions Generation</span>


In [6]:
num_interactions = 1_000_000

# Generate interactions
interactions = generate_interactions(num_interactions, user_data, video_data)

data_interactions_df = pd.DataFrame(interactions)
data_interactions_df.head()

,interaction_id,user_id,video_id,interaction_type,watch_time
0,2250-86-3648,OL411M,5LJ72P,skip,248
1,4882-58-7729,YR493Z,8XD91V,skip,129
2,8236-50-8032,DX952Q,3KV32I,skip,13
3,6091-83-5857,TP651F,9UU52V,view,19
4,9275-55-2106,GN960T,6QN75W,comment,166


## <span style="color:#ff5f27">👮🏻‍♂️ Great Expectations </span>

In [7]:
# Create a Great Expectations DataFrame from the pandas DataFrame
ge_users_df = ge.from_pandas(data_users_df)

# Initialize the expectation suite
expectation_suite_users = ge_users_df.get_expectation_suite()
expectation_suite_users.expectation_suite_name = "user_data_suite"

# Expectation: Age should be between 12 and 120
expectation_suite_users.add_expectation(
    ExpectationConfiguration(
        expectation_type="expect_column_values_to_be_between",
        kwargs={"column": "age", "min_value": 12, "max_value": 120}
    )
)

# Expectations: Columns should not have null values
for column in ge_users_df.columns:
    expectation_suite_users.add_expectation(
        ExpectationConfiguration(
            expectation_type="expect_column_values_to_not_be_null",
            kwargs={"column": column}
        )
    )

# Expectation: Gender should only contain specific values
expectation_suite_users.add_expectation(
    ExpectationConfiguration(
        expectation_type="expect_column_distinct_values_to_be_in_set",
        kwargs={"column": "gender", "value_set": ["Male", "Female", "Other"]}
    )
)

{"expectation_type": "expect_column_distinct_values_to_be_in_set", "kwargs": {"column": "gender", "value_set": ["Male", "Female", "Other"]}, "meta": {}}

In [8]:
# Create a Great Expectations DataFrame from the pandas DataFrame
ge_video_df = ge.from_pandas(data_video_df)

# Initialize the expectation suite
expectation_suite_videos = ge_video_df.get_expectation_suite()
expectation_suite_videos.expectation_suite_name = "video_data_suite"

# Expectation: Views, Likes, and Video Length should be non-negative
for column in ["views", "likes", "video_length"]:
    expectation_suite_videos.add_expectation(
        ExpectationConfiguration(
            expectation_type="expect_column_values_to_be_between",
            kwargs={"column": column, "min_value": 0, "max_value": None}
        )
    )

# Expectation: Valid date format for upload_date
expectation_suite_videos.add_expectation(
    ExpectationConfiguration(
        expectation_type="expect_column_values_to_be_dateutil_parseable",
        kwargs={"column": "upload_date"}
    )
)

{"expectation_type": "expect_column_values_to_be_dateutil_parseable", "kwargs": {"column": "upload_date"}, "meta": {}}

In [9]:
# Create a Great Expectations DataFrame from the pandas DataFrame
ge_interactions_df = ge.from_pandas(data_interactions_df)

# Initialize the expectation suite
expectation_suite_interactions = ge_interactions_df.get_expectation_suite()
expectation_suite_interactions.expectation_suite_name = "interactions_data_suite"

# Expectations: Non-null values in all columns
for column in ge_interactions_df.columns:
    expectation_suite_interactions.add_expectation(
        ExpectationConfiguration(
            expectation_type="expect_column_values_to_not_be_null",
            kwargs={"column": column}
        )
    )

# Expectation: Validate interaction types
expectation_suite_interactions.add_expectation(
    ExpectationConfiguration(
        expectation_type="expect_column_distinct_values_to_be_in_set",
        kwargs={
            "column": "interaction_type",
            "value_set": ['like', 'dislike', 'view', 'comment', 'share', 'skip']
        }
    )
)

# Expectation: Positive watch time
expectation_suite_interactions.add_expectation(
    ExpectationConfiguration(
        expectation_type="expect_column_values_to_be_between",
        kwargs={"column": "watch_time", "min_value": 0}
    )
)

{"expectation_type": "expect_column_values_to_be_between", "kwargs": {"column": "watch_time", "min_value": 0}, "meta": {}}

## <span style="color:#ff5f27">🔮 Connect to Hopsworks Feature Store </span>

In [10]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

Copy your Api Key (first register/login): https://c.app.hopsworks.ai/account/api/generated

Paste it here: ··········

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1196184


## <span style="color:#ff5f27">🪄 Feature Group Creation </span>


In [11]:
users_fg = fs.get_or_create_feature_group(
    name="users",
    description="Users data.",
    version=1,
    primary_key=["user_id"],
    online_enabled=True,
    expectation_suite=expectation_suite_users,
)

users_fg.insert(data_users_df)
print('Done ✅')

Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1196184/fs/1183821/fg/1380739


Uploading Dataframe: 100.00% |██████████| Rows 25000/25000 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: users_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1196184/jobs/named/users_1_offline_fg_materialization/executions
Done ✅


In [12]:
videos_fg = fs.get_or_create_feature_group(
    name="videos",
    description="Videos data.",
    version=1,
    primary_key=["video_id"],
    online_enabled=True,
    expectation_suite=expectation_suite_videos,
)

videos_fg.insert(data_video_df)
print('Done ✅')

Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1196184/fs/1183821/fg/1380741


Uploading Dataframe: 100.00% |██████████| Rows 25000/25000 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: videos_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1196184/jobs/named/videos_1_offline_fg_materialization/executions
Done ✅


In [13]:
interactions_fg = fs.get_or_create_feature_group(
    name="interactions",
    description="Interactions data.",
    version=1,
    primary_key=["interaction_id", "user_id", "video_id"],
    online_enabled=True,
    expectation_suite=expectation_suite_interactions,
)

interactions_fg.insert(data_interactions_df)
print('Done ✅')

Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1196184/fs/1183821/fg/1381734


Uploading Dataframe: 100.00% |██████████| Rows 1000000/1000000 | Elapsed Time: 00:40 | Remaining Time: 00:00


Launching job: interactions_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1196184/jobs/named/interactions_1_offline_fg_materialization/executions
Done ✅


## <span style="color:#ff5f27">🪄 Ranking Feature Group </span>


In [15]:
video_interactions_df = pd.merge(
    data_interactions_df,
    data_video_df,
    on='video_id',
    how='inner',
)

ranking_df = pd.merge(
    video_interactions_df,
    data_users_df,
    on='user_id',
    how='inner',
)

ranking_df['label'] = np.where(
    ranking_df.interaction_type.isin(['view', 'like', 'share', 'comment']),
    1,
    0,
)

ranking_df.drop(
    ['interaction_id', 'interaction_type', 'watch_time'],
    axis=1,
    inplace=True,
)

ranking_df.sample(10)

,user_id,video_id,category,views,likes,video_length,upload_date,gender,age,country,label
550315,AN112F,2MB79S,Lifestyle,320847,220531,135,2024-07-27,Other,21,Lithuania,0
241197,TS396J,5EE11F,Education,29021,19887,42,2023-08-21,Other,33,Antigua & Barbuda,1
456374,CK804M,4GN21T,Dance,25868,22638,99,2023-03-31,Male,64,Cameroon,0
898098,NW945R,6LS69P,Cooking,129573,73491,36,2023-10-19,Female,70,British Indian Ocean Territory,0
267277,PI090L,9IP57U,Lifestyle,58173,5903,205,2024-05-16,Other,71,Spain,0
924116,GU408N,3AS59O,Dance,52228,48712,212,2024-03-23,Male,31,Aruba,0
223731,OX908W,4EM52Q,Entertainment,196007,151488,78,2024-12-09,Male,80,Mauritania,1
512412,FH841U,7TA08X,Entertainment,26281,17821,15,2024-08-02,Female,74,Bolivia,0
593982,FQ837J,5DV28W,Education,45333,34237,42,2023-03-15,Female,38,Switzerland,1
588796,WV081K,4GS94B,Music,70729,50958,210,2023-06-29,Other,57,Bahamas,0


In [16]:
ranking_fg = fs.get_or_create_feature_group(
    name="ranking",
    description="Ranking Data.",
    version=1,
    primary_key=["user_id", "video_id"],
    online_enabled=True,
)

ranking_fg.insert(ranking_df)
print('Done ✅')

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1196184/fs/1183821/fg/1381737


Uploading Dataframe: 100.00% |██████████| Rows 1002889/1002889 | Elapsed Time: 01:04 | Remaining Time: 00:00


Launching job: ranking_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1196184/jobs/named/ranking_1_offline_fg_materialization/executions
Done ✅


---